In [15]:
import sys, os
repo_root = "/home/agnes/Skyclean"          # <- adjust if different
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

from skyclean.silc.pipeline import Pipeline

# confirm
import skyclean, skyclean.silc.pipeline
print("skyclean from:", skyclean.__file__)
print("pipeline  from:", skyclean.silc.pipeline.__file__)


skyclean from: /home/agnes/Skyclean/skyclean/__init__.py
pipeline  from: /home/agnes/Skyclean/skyclean/silc/pipeline.py


In [16]:
import sys, jax
print(sys.executable)      # should be .../miniconda3/envs/clean_cmb/bin/python
print(jax.__version__)     # 0.5.x
print(jax.devices())       # [CudaDevice(...)]

/home/agnes/miniconda3/envs/clean_cmb/bin/python
0.5.3
[CudaDevice(id=0), CudaDevice(id=1)]


In [17]:
pipe = Pipeline(
    components=["cmb", "sync", "tsz", "noise"],
    wavelet_components=["cfn"],
    ilc_components=["cmb"],   # extract CMB via (c)ILC
    frequencies=["030","044","070","100","143","217","353","545","857"],
    realisations=1,           # r00000 only; use 300 for r00000..r00299
    start_realisation=0,
    lmax=512,
    N_directions=1,
    lam=2.0,
    method="jax",
    save_ilc_intermediates=True,
    overwrite=False,    
    visualise=True,      
    directory="/Scratch/agnes/data",
)

pipe.run()

No steps selected. Use --steps with any of: download process wavelets ilc all


In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

pipe = Pipeline(
    components=["cmb","sync","noise","tsz"],
    wavelet_components=["cfn"],
    ilc_components=["cfn"],
    
    frequencies=["030","044","070","100","143","217","353","545","857"],
    realisations=1,
    start_realisation=0,
    lmax=32,
    N_directions=1,
    lam=2.0,
    method="jax_cuda",
    visualise=True,
    save_ilc_intermediates=False,
    overwrite=False,
    directory="/Scratch/agnes/data",
)

In [19]:
pipe.step_download()

--- STARTING DATA DOWNLOAD ---
File /Scratch/agnes/data/CMB_realisations/sync_f030.fits already exists. Skipping download.
File /Scratch/agnes/data/CMB_realisations/sync_f044.fits already exists. Skipping download.
File /Scratch/agnes/data/CMB_realisations/sync_f070.fits already exists. Skipping download.
File /Scratch/agnes/data/CMB_realisations/sync_f100.fits already exists. Skipping download.
File /Scratch/agnes/data/CMB_realisations/sync_f143.fits already exists. Skipping download.
File /Scratch/agnes/data/CMB_realisations/sync_f217.fits already exists. Skipping download.
File /Scratch/agnes/data/CMB_realisations/sync_f353.fits already exists. Skipping download.
File /Scratch/agnes/data/CMB_realisations/sync_f545.fits already exists. Skipping download.
File /Scratch/agnes/data/CMB_realisations/sync_f857.fits already exists. Skipping download.
File /Scratch/agnes/data/CMB_realisations/tsz_f030.fits already exists. Skipping download.
File /Scratch/agnes/data/CMB_realisations/tsz_f044

In [20]:
pipe.step_process()

--- PROCESSING CFNs AND TOTAL MAP CFN ---
CFN map at 030 GHz for realisation 0 already exists. Skipping processing.
CFN map at 044 GHz for realisation 0 already exists. Skipping processing.
CFN map at 070 GHz for realisation 0 already exists. Skipping processing.
CFN map at 100 GHz for realisation 0 already exists. Skipping processing.


setting the output map dtype to [dtype('float64')]
setting the output map dtype to [dtype('float64')]
setting the output map dtype to [dtype('float64')]


CFN map at 143 GHz for realisation 0 saved to /Scratch/agnes/data/CFN_realisations/cfn_f143_r0000_lmax32.npy


setting the output map dtype to [dtype('float64')]
setting the output map dtype to [dtype('float64')]
setting the output map dtype to [dtype('float64')]
setting the output map dtype to [dtype('float64')]


CFN map at 217 GHz for realisation 0 saved to /Scratch/agnes/data/CFN_realisations/cfn_f217_r0000_lmax32.npy


setting the output map dtype to [dtype('float64')]


KeyboardInterrupt: 

In [ ]:
pipe.step_wavelets()

--- PRODUCING WAVELET TRANSFORMS ---
Wavelet coefficients for cfn at 030 GHz r0 exist. Skipping.
Wavelet transform for cfn at 030 GHz for realisation 0 saved.
Wavelet coefficients for cfn at 044 GHz r0 exist. Skipping.
Wavelet transform for cfn at 044 GHz for realisation 0 saved.
Wavelet coefficients for cfn at 070 GHz r0 exist. Skipping.
Wavelet transform for cfn at 070 GHz for realisation 0 saved.
Wavelet coefficients for cfn at 100 GHz r0 exist. Skipping.
Wavelet transform for cfn at 100 GHz for realisation 0 saved.
Wavelet coefficients for cfn at 143 GHz r0 exist. Skipping.
Wavelet transform for cfn at 143 GHz for realisation 0 saved.
Wavelet coefficients for cfn at 217 GHz r0 exist. Skipping.
Wavelet transform for cfn at 217 GHz for realisation 0 saved.
Wavelet coefficients for cfn at 353 GHz r0 exist. Skipping.
Wavelet transform for cfn at 353 GHz for realisation 0 saved.
Wavelet coefficients for cfn at 545 GHz r0 exist. Skipping.
Wavelet transform for cfn at 545 GHz for realisat

In [ ]:
pipe.step_ilc()

--- RUNNING ILC (new functional API) ---
--- PRODUCING ILC FOR target='cfn' (source on disk='cmb'), scales=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] ---
Processing realisation 0000 for component cmb


2025-08-24 17:03:24.095329: W external/xla/xla/stream_executor/cuda/cuda_command_buffer.cc:668] Retry CUDA graph instantiation after OOM error
E0824 17:03:24.095450 1021416 pjrt_stream_executor_client.cc:3077] Execution of replica 0 failed: RESOURCE_EXHAUSTED: Underlying backend ran out of memory trying to instantiate graph with 6 nodes and 0 conditionals (total of 0 alive graphs in the process). You can try to (a) Give more memory to the driver by reducing XLA_CLIENT_MEM_FRACTION (b) Disable command buffers with 'XLA_FLAGS=--xla_gpu_enable_command_buffer=' (empty set). Original error: Failed to instantiate CUDA graph: CUDA_ERROR_OUT_OF_MEMORY: out of memory


XlaRuntimeError: RESOURCE_EXHAUSTED: Underlying backend ran out of memory trying to instantiate graph with 6 nodes and 0 conditionals (total of 0 alive graphs in the process). You can try to (a) Give more memory to the driver by reducing XLA_CLIENT_MEM_FRACTION (b) Disable command buffers with 'XLA_FLAGS=--xla_gpu_enable_command_buffer=' (empty set). Original error: Failed to instantiate CUDA graph: CUDA_ERROR_OUT_OF_MEMORY: out of memory